In [36]:
import fastf1
import plotly.express as px
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [20]:
session = fastf1.get_session(2021, 'Monaco', 'R')
session

2021 Season Round 5: Monaco Grand Prix - Race

In [21]:
session.load()

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.0.4]
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written 

In [120]:
big_laps = pd.read_csv("../raw_data/laps.csv")

#fastf1.get_event_schedule(2017)
#big_laps['Compound'].fillna(method="bfill",limit=3,inplace=True)

big_laps[['Compound','Year',"LapNumber"]][big_laps['Compound'].isna() & big_laps['LapNumber']==1]

/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/655821644.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  big_laps = pd.read_csv("../raw_data/laps.csv")


,Compound,Year,LapNumber
0,NaN,2018,1.0
14,NaN,2018,1.0
72,NaN,2018,1.0
130,NaN,2018,1.0
188,NaN,2018,1.0
...,...,...,...
17636,NaN,2018,1.0
17688,NaN,2018,1.0
17741,NaN,2018,1.0
17794,NaN,2018,1.0


In [66]:
start_2023 = big_laps[(big_laps['LapNumber']==1) & (big_laps['Year']==2018)]
start_before = big_laps[(big_laps['LapNumber']==2) & (big_laps['Year']<2023)]
big_laps.columns
#plt.hist(start_2023['Compound'])
#plt.hist(start_before['Compound'])

Index(['Unnamed: 0', 'Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber',
       'Stint', 'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time',
       'Sector3Time', 'Sector1SessionTime', 'Sector2SessionTime',
       'Sector3SessionTime', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST',
       'IsPersonalBest', 'Compound', 'TyreLife', 'FreshTyre', 'Team',
       'LapStartTime', 'LapStartDate', 'TrackStatus', 'Position', 'Deleted',
       'DeletedReason', 'FastF1Generated', 'IsAccurate', 'Location', 'Year'],
      dtype='object')

In [155]:
pd.set_option('display.max_columns', None)

big_laps["second_compound"]=False
big_laps["second_compound"][big_laps['Stint']==1] = False

norris = big_laps[(big_laps['Driver']=="NOR")]


norris = norris[['Driver','LapNumber','Compound','Location','Year']][(norris.Location=="Barcelona") & (norris.Year==2023)]
#norris["second_compound"] = norris.apply(lambda x: 
#len(norris['Compound'].unique())

norris["prev_compound"]=norris["Compound"].shift(1)

big_laps = big_laps[(big_laps.Location=="Barcelona") & (big_laps.Year==2023)]
years = big_laps['Year'].unique()
for year in years:
    year_df = big_laps.loc[big_laps.Year == year]
    locations = year_df['Location'].unique()
    for location in locations:
        loc_df = year_df.loc[(year_df.Location == location)]
        drivers = loc_df['DriverNumber'].unique()
        for driver in drivers:
            driver_df = loc_df.loc[(loc_df.DriverNumber == driver)]
            driver_df["prev_compound"]=driver_df["Compound"].shift(1)
            #driver_df["prev_second_compound"]=driver_df["second_compound"].shift(1)
            #print(driver_df[['Driver','LapNumber','Compound','Location','Year','prev_compound']]
            first_index = driver_df.index[0]
            for index, row in driver_df.iterrows():
                if index-1>=first_index:
                    if driver_df.loc[index-1,"second_compound"]==True :
                        print("Hello1")
                        driver_df.loc[index,"second_compound"]=True
                        big_laps.loc[index,"second_compound"]=True
                elif row["Compound"] != ["prev_compound"] and row["prev_compound"]:
                    print("Hello2")
                    print(row)
                    driver_df.loc[index,"second_compound"]=True
                    big_laps.loc[index,"second_compound"]=True
                    print(driver_df.loc[index,"second_compound"])
                else:
                    driver_df.loc[index,"second_compound"]=False
                    big_laps.loc[index,"second_compound"]=False

big_laps[big_laps.Driver=="VER"]

/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/1880961848.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_laps["second_compound"][big_laps['Stint']==1] = False
/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/1880961848.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_df["prev_compound"]=driver_df["Compound"].shift(1)
/var/folders/7r/dg7644r91fsfzzv3xfr9pzt80000gn/T/ipykernel_56522/1880961848.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

,Unnamed: 0,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Location,Year,second_compound
124447,124447,0 days 01:03:40.152000,VER,1,NaN,1.0,1.0,0 days 00:13:44.843000,NaN,NaN,0 days 00:00:32.084000,0 days 00:00:23.926000,NaN,0 days 01:03:16.243000,0 days 01:03:40.243000,256.0,261.0,276.0,275.0,False,MEDIUM,1.0,True,Red Bull Racing,0 days 01:02:15.963000,2023-06-04 13:03:15.970,1.0,1.0,False,NaN,False,False,Barcelona,2023,False
124448,124448,0 days 01:05:00.554000,VER,1,0 days 00:01:20.402000,2.0,1.0,NaN,NaN,0 days 00:00:24.186000,0 days 00:00:32.088000,0 days 00:00:24.128000,0 days 01:04:04.338000,0 days 01:04:36.426000,0 days 01:05:00.554000,252.0,257.0,276.0,295.0,True,MEDIUM,2.0,True,Red Bull Racing,0 days 01:03:40.152000,2023-06-04 13:04:40.159,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124449,124449,0 days 01:06:21.053000,VER,1,0 days 00:01:20.499000,3.0,1.0,NaN,NaN,0 days 00:00:24.167000,0 days 00:00:32.191000,0 days 00:00:24.141000,0 days 01:05:24.721000,0 days 01:05:56.912000,0 days 01:06:21.053000,249.0,256.0,276.0,297.0,False,MEDIUM,3.0,True,Red Bull Racing,0 days 01:05:00.554000,2023-06-04 13:06:00.561,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124450,124450,0 days 01:07:41.399000,VER,1,0 days 00:01:20.346000,4.0,1.0,NaN,NaN,0 days 00:00:24.022000,0 days 00:00:32.159000,0 days 00:00:24.165000,0 days 01:06:45.075000,0 days 01:07:17.234000,0 days 01:07:41.399000,255.0,256.0,276.0,300.0,True,MEDIUM,4.0,True,Red Bull Racing,0 days 01:06:21.053000,2023-06-04 13:07:21.060,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124451,124451,0 days 01:09:01.682000,VER,1,0 days 00:01:20.283000,5.0,1.0,NaN,NaN,0 days 00:00:24.034000,0 days 00:00:32.213000,0 days 00:00:24.036000,0 days 01:08:05.433000,0 days 01:08:37.646000,0 days 01:09:01.682000,254.0,256.0,277.0,301.0,True,MEDIUM,5.0,True,Red Bull Racing,0 days 01:07:41.399000,2023-06-04 13:08:41.406,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124508,124508,0 days 02:25:02.582000,VER,1,0 days 00:01:18.014000,62.0,3.0,NaN,NaN,0 days 00:00:23.411000,0 days 00:00:31.301000,0 days 00:00:23.302000,0 days 02:24:07.979000,0 days 02:24:39.280000,0 days 02:25:02.582000,264.0,270.0,284.0,NaN,False,SOFT,12.0,False,Red Bull Racing,0 days 02:23:44.568000,2023-06-04 14:24:44.575,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124509,124509,0 days 02:26:20.279000,VER,1,0 days 00:01:17.697000,63.0,3.0,NaN,NaN,0 days 00:00:23.214000,0 days 00:00:31.199000,0 days 00:00:23.284000,0 days 02:25:25.796000,0 days 02:25:56.995000,0 days 02:26:20.279000,270.0,267.0,286.0,310.0,False,SOFT,13.0,False,Red Bull Racing,0 days 02:25:02.582000,2023-06-04 14:26:02.589,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124510,124510,0 days 02:27:38.838000,VER,1,0 days 00:01:18.559000,64.0,3.0,NaN,NaN,0 days 00:00:23.093000,0 days 00:00:31.812000,0 days 00:00:23.654000,0 days 02:26:43.372000,0 days 02:27:15.184000,0 days 02:27:38.838000,263.0,238.0,284.0,NaN,False,SOFT,14.0,False,Red Bull Racing,0 days 02:26:20.279000,2023-06-04 14:27:20.286,1.0,1.0,False,NaN,False,True,Barcelona,2023,False
124511,124511,0 days 02:28:56.449000,VER,1,0 days 00:01:17.611000,65.0,3.0,NaN,NaN,0 days 00:00:23.162000,0 days 00:00:31.116000,0 days 00:00:23.333000,0 days 02:28:02,0 days 02:28:33.116000,0 days 02:28:56.449000,273.0,273.0,284.0,307.0,False,SOFT,15.0,False,Red Bull Racing,0 days 02:27:38.838000,2023-06-04 14:28:38.845,1.0,1.0,False,NaN,False,True,Barcelona,2023,False


In [52]:
start_before = big_laps[(big_laps['LapNumber']==4)]
start_before.columns

Index(['Unnamed: 0', 'Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber',
       'Stint', 'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time',
       'Sector3Time', 'Sector1SessionTime', 'Sector2SessionTime',
       'Sector3SessionTime', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST',
       'IsPersonalBest', 'Compound', 'TyreLife', 'FreshTyre', 'Team',
       'LapStartTime', 'LapStartDate', 'TrackStatus', 'Position', 'Deleted',
       'DeletedReason', 'FastF1Generated', 'IsAccurate', 'Location', 'Year'],
      dtype='object')

In [40]:
import math
import numpy as np

#def track_angle(prev_x, prev_y, curr_x, curr_y, next_x, next_y):
def track_angle(row):
    print(row)
    a = np.array([row['X_prev'],row['Y_prev']])
    b = np.array([row['X'],row['Y']])
    c = np.array([row['X_next'],row['Y_next']])

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    
    return np.degrees(angle)

In [13]:
gasly = session.get_driver("OCO")
#weather_data = session.weather_data
#control_msg = session.race_control_messages

def add_standardized_time(df):
    df['Time_min']=df['Time'].astype('timedelta64[m]')

miami_laps = session.laps
tele = session.get_driver("OCO")
gasly = miami_laps.pick_driver('OCO')
gasly_car = gasly.get_car_data()
gasly_pos = gasly.get_pos_data()
prev = gasly_pos.shift(1)
next = gasly_pos.shift(-1)
gasly_pos = gasly_pos.merge(prev,on="Date",how="left",suffixes=(None,"_prev"))
gasly_pos = gasly_pos.merge(next,on="Date",how="left",suffixes=(None,"_next"))
gasly_data = gasly_car.copy()
gasly_pos.info()
add_standardized_time(gasly_data)
add_standardized_time(weather_data)
#gasly_data['Time_min']=gasly_data['Time'].astype('timedelta64[m]')
#weather_data['Time_min']=weather_data['Time'].astype('timedelta64[m]')
gasly_data = gasly_data.merge(weather_data,on="Time_min",how="left")
#gasly_data = gasly_data.merge(gasly_pos, on="Time",how="left")
control_msg['Date']=control_msg['Time']
#gasly_data = gasly_data.merge(control_msg,on="Date",how="left")
gasly_data.to_csv("test.csv")
#gasly_pos['angle'] = gasly_pos.apply(lambda x: track_angle(x))

gasly_pos


DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`

In [7]:
fig = px.scatter(x=gasly_pos['X'],y=gasly_pos['Y'])#,animation_frame=gasly_pos.index,animation_group=gasly_pos['Source'])
fig.show()